### Import and Clean Data Script

#### Author: Lauren Thomas
#### Created: 01/05/2021
#### Last updated: 30/06/2021

###### File description: This file imports, cleans and pre-processes the data that will be used in the ML models.

In [ ]:
import os
import gzip
import glob
import json
import pickle

import geopandas as gp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import osmium as osm
from os import sep
from shapely.geometry import Point,Polygon,MultiPolygon

In [ ]:
# Working directory
cwd = f"C:{sep}Users{sep}ltswe{sep}Dropbox{sep}Oxford{sep}Thesis"
# Data directory is kept on flash
data_dir = "D:"

#### Assign Census Tracts using Shapefiles

In [ ]:
# Read in NYC Census Tract files
nycct = gp.read_file(f'{cwd}{sep}data{sep}shapefiles{sep}nyct2010.shp')

In [ ]:
# Project shapefile into lat/long
nycct_proj = nycct.to_crs("epsg:4326")

# Generate full geoids (state FIPS + county FIPS + CT FIPS)
# NY State FIPS: 36
# County fips: 061 (Manhattan), 005 (Bronx), 081 (Queens), 085 (Staten Island), 047 (Brooklyn).
# BoroCT2010 uses a weird county code system where 1=Manhattan,2=Bronx,3=Brooklyn,4=Queens,& 5=SI
def gen_fips(x):
    ''' x = BoroCT2010 code'''
    county_code = x[0:1]
    ct_code = x[1:7]
    if county_code == '5': #SI
        return '36'+'085' + ct_code
    elif county_code == '1': #Manhattan
        return '36'+'061' + ct_code
    elif county_code == '4': #Queens
        return '36'+'081' + ct_code
    elif county_code == '3': #Brooklyn
        return '36'+'047' + ct_code
    else: #Bronx
        return '36'+'005' +ct_code

nycct_proj['fips_code'] = nycct_proj['BoroCT2010'].apply(lambda x: gen_fips(x))

In [ ]:
nycct_proj.plot()

In [ ]:
# Create spatial index for R-trees for tweets
tweet_sindex = tweet_df.sindex

In [ ]:
print("Number of groups:", len(tweet_sindex.leaves()), '\n')

In [ ]:
# Create a list of census tracts to iterate through
tracts = list(nycct_proj.fips_code)


# Iterate through tracts & find all the tweets that correspond to each given tract
i = 1
for tract in tracts:
    select_tract = nycct_proj.loc[nycct_proj['fips_code']==tract]
    
#     # plot relevant census tract
#     ax = select_tract.plot(color = 'red', alpha = 0.5)
#     plt.show()
    
    # Get the bounding box coordinates of the census tract as a list
    bounds = list(select_tract.bounds.values[0])
    
    #Get the indices of the points that are inside the bounding box of the given census tract
    tweet_candidate_idx = list(tweet_sindex.intersection(bounds))
    tweet_candidates = tweet_df.loc[tweet_candidate_idx]


#     # Let's see what we have now
#     ax = select_tract.plot(color='red', alpha = 0.5)
#     ax = tweet_candidates.plot(ax=ax, color='blue', markersize = 2)
#     plt.show()
    

    # Now make the precise Point in Polygon query
    tweet_final_selection = tweet_candidates.loc[tweet_candidates.intersects(select_tract['geometry'].values[0])]
    
    
#     # Let's see what we have now
#     ax = select_tract.plot(color='red', alpha = 0.5)
#     ax = tweet_final_selection.plot(ax=ax, color='blue', markersize = 2)
#     plt.show()
    
    # Put correct tract back in original DF
    tweet_df.loc[list(tweet_final_selection.index.values),'LocationCT'] = tract
    
    i+= 1
    if i%100 == 0:
        print(f'another hundred done! up to {i} census tracts')
    
# Drop any tweets that are outside NYC
tweet_df = tweet_df[tweet_df['LocationCT'].isna() == False].reset_index(drop=True)
display(tweet_df)

In [ ]:
tweet_df.plot()

#### Import & preprocess Tweet data

In [ ]:
# Restructure JSONs into JSOLs (where each line = one tweet) for each month-year from Jan 2007 to Dec 2013
# make list of month-year pairs
ym_list = [str(year)+"-"+("0"+str(month))[-2:] for month in range(1,13) for year in range(2007,2014)]

# Create a dictionary that will contain all the JSONs for a given list of month-year pairs

def create_json(ym_list, json_pickle_str):
    all_jsons = dict()
    for ym in ym_list:
        print(ym)
        # Create a list of the jsons that fall into that y-m - excluding all outputs that ends in 00000.json.
        json_list = [j for j in glob.glob(f'{data_dir}{sep}raw_tweets{sep}{ym}*{sep}*.json', recursive=True) 
                 if j[-10:] != '00000.json']
        # Create list of JSONs that we will append to the larger dictionary 
        temp_json_list = list()
        for j in json_list:
            temp_json = json.load(open(j, encoding = 'utf-8'))['data']
            temp_json_list.extend(temp_json)
        # Add temp_dict to larger dictionary of all JSONs with the key as the year-month
        all_jsons[ym] = temp_json_list
    # Pickle JSON
    tweets_json_pickle = open(f"{data_dir}{sep}pickle{sep}{json_pickle_str}.pickle", "wb")
    pickle.dump(all_jsons, tweets_json_pickle)


In [ ]:
# Create JSONs in 12 chunks (one for each month)
# json_01 = create_json(ym_list[0:7], 'json_01')
# json_02 = create_json(ym_list[7:14], 'json_02')
# json_03 = create_json(ym_list[14:21], 'json_03')
# json_04 = create_json(ym_list[21:28], 'json_04')
# json_05 = create_json(ym_list[28:35], 'json_05')
# json_06 = create_json(ym_list[35:42], 'json_06')
# json_07 = create_json(ym_list[42:49], 'json_07')
# json_08 = create_json(ym_list[49:56], 'json_08')
# json_09 = create_json(ym_list[56:63], 'json_09')
# json_10 = create_json(ym_list[63:70], 'json_10')
# json_11 = create_json(ym_list[70:77], 'json_11')
# json_12 = create_json(ym_list[77:84], 'json_12')

In [ ]:
def unpickle_json(num):
    ''' This function unpickles the relevant JSON'''
    return pickle.load(open(f"{data_dir}{sep}pickle{sep}json_{num}.pickle", "rb"))

In [ ]:
json_03 = unpickle_json("03")

In [ ]:
json_03.keys()

In [ ]:
# author_id, id, lang, public_metrics.like_count, public_metrics.quote_count, public_metrics.reply_count,
# public_metrics.retweet_count, text
ym_list_test = ['2007-03', '2008-03','2009-03', '2010-03', '2011-03', '2012-03', '2013-03']

In [ ]:
json = json_03
x_list, y_list, date_list,geo_list, author_list, tweet_list, lang_list, like_list, quote_list, reply_list, retweet_list, \
text_list, ym_list = list(),list(),list(),list(),list(),list(),list(),list(),list(),list(),list(),list(),list()
# Make lists of important things 
for ym in ym_list_test:
    print(ym)
    for j in json[ym]:
        try:
            geo_id = Point(j["geo"]["coordinates"]["coordinates"])
            # Annoyingly, some of the tweets are geotagged outside NYC. Get rid of these.
            # These lat/long maxes and mins are taken from the NYC census tract shapefiles. 
            # Note x is long and y is lat (thanks, Twitter!)
            if geo_id.x <= -73.70000924132164 and geo_id.x >= -74.25559213002796 \
            and geo_id.y <= 40.91553243056209 and geo_id.y >= 40.49611511946593: 
                date_list.append(j['created_at'])
                x_list.append(geo_id.x)
                y_list.append(geo_id.y)
                geo_list.append(geo_id), author_list.append(j['author_id']), tweet_list.append(j['id'])
                lang_list.append(j['lang']), like_list.append(j['public_metrics']['like_count'])
                quote_list.append(j['public_metrics']['quote_count']), reply_list.append(j['public_metrics']['reply_count']) 
                retweet_list.append(j['public_metrics']['retweet_count']), text_list.append(j['text'])
                ym_list.append(ym)
        except KeyError:
            continue
# Create dataframe for month 
tweet_df = gp.GeoDataFrame(
    {'ym': ym_list,
     'date':date_list,
    'tweet_id':tweet_list,
     'author_id':author_list,
    'lang': lang_list,
    'like_count': like_list,
     'quote_count': quote_list,
     'reply_count': reply_list,
     'retweet_count': retweet_list,
     'text': text_list,
     'x': x_list,
     'y': y_list,
     'geometry': geo_list
    }
)

#### Import and preprocess other data

In [ ]:
# Bring in crime and 311 data, which uses the Socrata API in NYC Open Data
# Create a function that uses the Socrata API, which is written in SoQL, a SQL-like language, to query data
from sodapy import Socrata

def socrata_API_df(source_domain, dataset_id, select_string, where_string, limit=1000):
    '''
    Inputs: 
    source_domain: This tells Socrata the source of the dataset you're querying
    dataset_id: This is the unique id of the dataset
    select_string: This string tells Socrata which variables you are selecting from the dataset
    where_string: This string is equivalent to the "where" command in SQL
    limit = This tells Socrata how many results to query. The default is 1000 b/c Socrata automatically sets it to 1000

    Outputs a dataframe with with the queried results
    '''
    keyFile = open(f'{cwd}{sep}tokens{sep}socrata_apikey.txt', 'r')
    token = keyFile.readline() #api token imported from txt file
    
    client = Socrata(source_domain, token)
    # Change timeout var to arbitrarily large # of seconds so it doesn't time out
    client.timeout = 50
    results = client.get(dataset_id, limit = limit, select = select_string, where = where_string)
    df = pd.DataFrame.from_records(results)
    return df


In [ ]:
# Pull in 311 and Null Data 
# 2007, 2008, & 2009 are separate; 2010-on are in a single file. 
# The only thing that changes between 2007-09 is the dataset ID, & the id + where string for 2010-on
# so write a function that calls upon the 311 socrata API data
# complaint type string -- separated for ease of understanding. Complaint types drawn from literature
complaint_type_str = "complaint_type = 'Noise - Street/Sidewalk' OR complaint_type = 'Noise - Residential' OR complaint_type = 'Noise - Vehicle' OR complaint_type = 'Street Condition' " \
                    "OR complaint_type = 'Homeless Encampment' OR complaint_type = 'Drinking' OR complaint_type = 'Noise' " \
                    "OR complaint_type = 'Noise - Park' OR complaint_type = 'Noise - House of Worship' OR complaint_type = 'HEATING' " \
                    "OR complaint_type = 'GENERAL CONSTRUCTION' OR complaint_type = 'CONSTRUCTION' OR complaint_type = 'Boilers' " \
                    "OR complaint_type = 'For Hire Vehicle Complaint' OR complaint_type = 'Bike Rack Condition' OR complaint_type = 'Illegal Parking' " \
                    "OR complaint_type = 'Building/Use' OR complaint_type = 'ELECTRIC' OR complaint_type = 'PLUMBING'"

def pull_311(dataset_id, where_string = f'latitude IS NOT NULL AND ({complaint_type_str})'):
    return socrata_API_df(source_domain = "data.cityofnewyork.us", dataset_id = dataset_id, \
                         select_string = 'unique_key, created_date, complaint_type, date_extract_y(created_date) as year, date_extract_m(created_date) as month, descriptor, latitude, longitude', \
                         where_string = where_string,
                         limit = 4000000)

# 2007-2013
nyc_311_07 = pull_311("aiww-p3af")
nyc_311_08 = pull_311('uzcy-9puk')
nyc_311_09 = pull_311('3rfa-3xsf')
nyc_311_10_13 = pull_311('erm2-nwe9', \
                where_string = f'({complaint_type_str}) AND latitude IS NOT NULL AND (year = 2010 OR year = 2011 OR year = 2012 OR year = 2013)')

# Combine all four
nyc_311 = nyc_311_07.append(nyc_311_08).append(nyc_311_09).append(nyc_311_10_13)


In [ ]:
nyc_311.complaint_type.unique()

In [ ]:
nyc_311_pickle = open(f"{data_dir}{sep}pickle{sep}nyc_311.pickle", "wb")
pickle.dump(nyc_311, nyc_311_pickle)

In [ ]:
# Pull in NYC historical crime data (also uses Socrata data)
select_string = 'cmplnt_num, cmplnt_fr_dt AS date, date_extract_y(cmplnt_fr_dt) AS year,' \
    'date_extract_m(cmplnt_fr_dt) AS month,  pd_cd AS class, pd_desc, law_cat_cd AS level, crm_atpt_cptd_cd AS completed, latitude, longitude'
where_string = 'latitude IS NOT NULL AND (year = 2007 OR year = 2008 OR year = 2009 OR year = 2010 OR year = 2011 OR year = 2012 OR year = 2013)'
nyc_crime = socrata_API_df(source_domain = "data.cityofnewyork.us", dataset_id = 'qgea-i56i', \
                           select_string = select_string, where_string = where_string, limit = 4000000)


In [ ]:
nyc_crime_pickle = open(f"{data_dir}{sep}pickle{sep}nyc_crime.pickle", "wb")
pickle.dump(nyc_crime, nyc_crime_pickle)

In [ ]:
# Bring in HUD vacant addresses data
# Create list of the excel files that will need to be loaded in 
# Glob.glob creates a list of all the files that end in .xlsx in the directory of HUD vacant data
# The rest of the command filters out jsons that end in 00000.json since those represent meta counts and not actual tweets
hud_list = [j for j in glob.glob(f'{data_dir}{sep}hud_vacant_data{sep}*.csv')]

In [ ]:
hud_df = pd.DataFrame()
for file in hud_list:
    temp_file = pd.read_csv(file, sep = None, engine='python')
#   Using title of the file, create a column for the year & month/quarter
    temp_file['year'] = ["20"+file[32:34] for i in range(temp_file.shape[0])]
    temp_file['month'] = [file[28:30] for i in range(temp_file.shape[0])]
    hud_df = hud_df.append(temp_file).reset_index(drop=True)


In [ ]:
# Create a FIPS code variable that is equal to the string of geoid 
# (note that b/c it's in integers, we need to re-add leading zero for states with fips codes < 10) 
hud_df['fips_code'] = hud_df["GEOID"].apply(lambda x: ("0" + str(x))[-11:])

# Create file with only NY 
ny_hud = hud_df[hud_df['fips_code'].apply(lambda x: x[0:2] == "36")].reset_index(drop=True)

# Pickle ny hug file
nyc_hud_pickle = open(f"{data_dir}{sep}pickle{sep}nyc_hud.pickle", "wb")
pickle.dump(ny_hud, nyc_hud_pickle)

In [ ]:
# Unpickle ny hud
ny_hud = pickle.load(open(f"{data_dir}{sep}pickle{sep}nyc_hud.pickle", "rb"))

In [ ]:
ny_hud

In [ ]:
raw_df = pickle.load(open(f'{data_dir}{sep}pickle{sep}raw_tweets_df.pickle', "rb"))

In [ ]:
raw_df.columns